# Regímenes (sub/edge/super) y complejidad LZC

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from horizonte.sch.cml import CML, CMLParams

def lzc_binary(seq):
    # Simple Lempel-Ziv complexity for binary sequence
    s = ''.join('1' if v else '0' for v in seq)
    i, k, l = 0, 1, 1
    c = 1
    n = len(s)
    while True:
        if s[i+k-1] == s[l+k-1]:
            k += 1
            if l+k > n: c += 1; break
        else:
            if k > 1:
                i += 1
                if i == l:
                    c += 1
                    l += k
                    if l+1 > n: break
                    i = 0; k = 1
            else:
                l += 1
                if l+1 > n: break
                i = 0; k = 1
    return c / (n/np.log2(n))

def episode_lzc(n=128, r=3.9, kappa=0.14, noise=0.001, seed=0, steps=800):
    cml = CML(CMLParams(n, r, kappa, noise, seed))
    m = []
    for t in range(steps):
        x = cml.step()
        m.append(float(x.mean()))
    # binarizar por mediana
    med = np.median(m)
    seq = [1 if v>med else 0 for v in m]
    lam = cml.lyapunov_max(T=100)
    return lzc_binary(seq), lam

kappas = np.linspace(0.05, 0.3, 10)
lzcs, lams = [], []
for k in kappas:
    lzc, lam = episode_lzc(kappa=float(k))
    lzcs.append(lzc); lams.append(lam)
kappas, lzcs, lams


In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,3))
ax[0].plot(kappas, lzcs, '-o'); ax[0].set_title('LZC vs kappa'); ax[0].set_xlabel('kappa'); ax[0].set_ylabel('LZC norm')
ax[1].plot(kappas, lams, '-o'); ax[1].set_title('Lyapunov vs kappa'); ax[1].set_xlabel('kappa'); ax[1].set_ylabel('lambda1')
plt.tight_layout(); plt.show()
